# Cross-Feeding Simulation
Here we will simulate cross-feeding between two species of bacteria, inspired by previous work by [Dukovski et al., 2021](https://www.nature.com/articles/s41596-021-00593-3). For this purpose we will simulate two strains of *E. coli*: one which cannot uptake lactose and one which cannot metabolize galactose - with lactose being the only environmental carbon source being lactose set to a chemostat.

In [1]:
from copy import deepcopy

import cobra.io
from cobra.io import write_sbml_model

from process_bigraph import Composite
from process_bigraph.composite import ProcessTypes

from cdFBA import register_types
from cdFBA.processes.dfba import DFBA, UpdateEnvironment, Chemostat, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_chemo_spec, model_from_file

from matplotlib import pyplot as plt
from pprint import pprint

import cProfile
import pstats
import io


<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


## Create Models
First we create E.coli models and save them in the sbml format. We start off with the `iAF1260` model for the K-12 *E. coli* strain, and use the knockout andCOBRApy

In [ ]:
E_no_galE = cobra.io.load_model("iAF1260")

In [ ]:
#create model dictionary
model_dict = {
    'deltaGal':'/Users/tasnifrahman/Research/cdFBA Test/E_no_galE.xml',
    'deltaLac':'/Users/tasnifrahman/Research/cdFBA Test/E_no_LCTStex.xml'
}
#list exchange reactions
exchanges = []

In [6]:
model = model_from_file('/Users/tasnifrahman/Research/cdFBA Test/E_no_galE.xml')

In [11]:
sorted([reaction.name for reaction in model.exchanges if reaction.id == 'LCTStex'])

[]

In [12]:
model.reactions

[<Reaction ACGAM1PPpp at 0x12cf88680>,
 <Reaction 2AGPA140tipp at 0x12cf88c00>,
 <Reaction EX_acser_e at 0x12cf88b50>,
 <Reaction 2AGPA141tipp at 0x12cf88cb0>,
 <Reaction 2AGPA160tipp at 0x12cf88d60>,
 <Reaction 3OAR180 at 0x12cf88e10>,
 <Reaction 2AGPA161tipp at 0x12cf88ec0>,
 <Reaction 3OAR181 at 0x12cf88f70>,
 <Reaction 2AGPA180tipp at 0x12cf89020>,
 <Reaction 3OAR40 at 0x12cf890d0>,
 <Reaction 2AGPA181tipp at 0x12cf89180>,
 <Reaction 3OAR60 at 0x12cf89230>,
 <Reaction 3OAR80 at 0x12cf892e0>,
 <Reaction 3OAS100 at 0x12cf89390>,
 <Reaction 2AGPE120tipp at 0x12cf89440>,
 <Reaction 3OAS120 at 0x12cf894f0>,
 <Reaction 3OAS121 at 0x12cf895a0>,
 <Reaction 3OAS140 at 0x12cf89650>,
 <Reaction 12DGR120tipp at 0x12cf89700>,
 <Reaction 12DGR140tipp at 0x12cf897b0>,
 <Reaction 3OAS141 at 0x12cf89860>,
 <Reaction 12DGR141tipp at 0x12cf89910>,
 <Reaction 2AGPE140tipp at 0x12cf899c0>,
 <Reaction 12DGR160tipp at 0x12cf89a70>,
 <Reaction 12DGR161tipp at 0x12cf89b20>,
 <Reaction 3OAS160 at 0x12cf89bd

In [13]:
E_no_LCTStex.reactions.LCTStex

Reaction identifier,LCTStex
Name,Lactose transport via diffusion (extracellular to periplasm)
Memory address,0x135269b20
Stoichiometry,lcts_e --> lcts_p Lactose C12H22O11 --> Lactose C12H22O11
GPR,b1377 or b2215 or b0241 or b0929
Lower bound,0
Upper bound,0


In [14]:
E_no_LCTStex.reactions.EX_lcts_e

Reaction identifier,EX_lcts_e
Name,Lactose exchange
Memory address,0x13526b330
Stoichiometry,lcts_e <=> Lactose C12H22O11 <=>
GPR,
Lower bound,-1.0
Upper bound,999999.0
